<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/ML2023/blob/main/PokemonClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar as bibliotecas

In [18]:
import pandas as pd
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import confusion_matrix as CM
import numpy as np
import matplotlib.pyplot as plt

#Preparar o Dataset


Ler o data set

In [3]:
pokemonDF = pd.read_csv("/content/pokemon.csv")

## Préprocessar

### Remover as entradas que não deveriam pertencer ao dataset.

Aqui removemos as linhas que o enunciado solicita que nós não utlizemos.





O que é uma pena, o conhecimento a priório de pokémon diz que além de fraqueza contra elétrico, pokémons aquaticos tem fraqueza contra grama. Essa seria um ótimo parametro para selecionar pokémons como Swampert, Wishcash e Seismictoad, cujos tipos secundários (terra) conferem imunidade ao tipo elétrico.

Além disso, discartar o tipo secundário, resulta em um dataset imperfeito, já que a ordem do tipo do pokémon é inter operavel, vide: https://www.reddit.com/r/pokemon/comments/b2zvom/primary_vs_secondary_types/

 Dois pokémons pode ter o mesmo tipo em ordem reversa, como sableye (Dark/Ghost) e Spiritomb (Ghost/Dark). Além disso, pokémons podem ter o tipo pertinente como o segundo tipo, que são desconsiderados.

Além disso temos um exemplo que pertence a ambas as classes, Bibarel, que tem ambos os tipos, que por nossa classificacao, pertenceria ao tipo normal, mas que igualmente poderia ser classificado como tipo agua. Por ele ser um outlier nesse aspecto, ele foi removido.

In [4]:
#remover bibarel
pokemonDF = pokemonDF.drop(pokemonDF[pokemonDF['pokedex_number'] == 400].index,)

In [5]:
# remover linhas "ilegais"
pokemonDF = pokemonDF.drop(["abilities", "against_bug", "against_dark",	"against_dragon",	"against_fairy",	"against_fight",	"against_fire",	"against_flying",	"against_ghost", "against_grass","against_ground","against_ice","against_normal","against_poison","against_psychic","against_rock", "against_steel", "against_water", "type2"], axis =1,errors='ignore')


Além disso, alguns dados são irrelevantes ou difíceis de serem utilizados. Alguns exemplos são nomes, classificacao e numero no pokedex. Nome e classificacao podem dar pistas por trás do tipo de pokemon, por exemplo o pokémon seaking contém a palavra "sea" no nome, e é classificado como o goldfish pokémon, mas esse tipo de processamento cai fora do escopo do trabalho. Quanto ao número e geracao, são apenas ordenacoes arbitrárias que näo tem muita relacao com o pokemon em si. Além disso, o campo altura, peso e gender ratio, apresentam linhas sem valor. Não acreditamos que esses valores sejam muito relevantes, logo descartamos las também.

In [6]:
pokemonDF = pokemonDF.drop(["classfication", "name", "japanese_name", "pokedex_number", "generation", "weight_kg", "height_m", "percentage_male"], axis = 1, errors='ignore')

Outras váriaveis como se um pokémon é lendario ou não, soma total de status, e altura provavlemente não são relevantes, mas na incerteza manteremos esses dados e faremos uma analise futura.
Algumas caracteristicas que nosso conhecimento prévio nos diz que provavelmente são bons indicadores para o tipo são a fraqueza a eleétrico, já que quase todo pokemon tipo agua tem, além de fatores como os stats do pokémon (vida, defesa etc) que suspeitamos que os pokémons tipo agua tenham mais vida e ataque especial, e os tipo normal ataque e velocidade.

vamos separar em dois dataset também. O exercicio pede um classifcador binário, normal ou água, mas poderiamos ter um terceiro fator, nenhum para os demais pokémons. Por isso vamos criar um o dataframe normal e agua, DFNW.

### Transformar rotulos em números
Para a matemática funcionar melhor precisamos transformar o atributo tipo em um valor númérico, Water = 1, Normal = -1, resto = 0.

In [7]:
pokemonDF.loc[~pokemonDF['type1'].isin(["water","normal"]), 'type1'] = 0
pokemonDF.loc[pokemonDF['type1'].isin(["water"]), 'type1'] = 1
pokemonDF.loc[pokemonDF['type1'].isin(["normal"]), 'type1'] = -1

In [8]:
#remover não tipos agua ou normal.
DFWN = pokemonDF.drop(pokemonDF[((pokemonDF['type1'] != 1) & (pokemonDF['type1'] != -1))].index,)

### Sampling
Vamos seprar um número de pokémons para usar como teste e outros para treinamento.

Cabe notar que existem mais geracoes de pokémons que presentes nesse data set, e portanto poderiamos testar com eles.

A possibilidade de realizar esse processo vai depender dos dados mais importantes. Por esse motivo nós NÃO vamos normalizar os dados, já que isso poderia resultar em incosintências.

In [9]:
#Faz um split do data frame

In [10]:
pokemonDFY = pokemonDF.loc[:,"type1"]
pokemonDFX = pokemonDF.drop(["type1"], axis =1,errors='ignore')

DFWNY      = DFWN.loc[:,"type1"]
DFWNX      = DFWN.drop(["type1"], axis =1,errors='ignore')

In [11]:
x_train, x_test, y_train, y_test = tts(DFWNX, DFWNY, test_size=0.25, random_state=0)
#put y in a sklearn compatible type
y_train = y_train.astype('int')

## Analise dos dados

In [35]:
#rawData = pokemonDFX.loc[:].values
#rawData
#PCA().fit(X=rawData)

#Aprendizado

## Logistic Regression

In [28]:
lr = LogisticRegression()
lr.fit(x_train.values, y_train.values)
lr_predicts = lr.predict(x_test.values)
y_test = y_test.astype('int')

CM(lr_predicts, y_test)

array([[21, 12],
       [ 5, 17]])

## SVM

In [29]:
svm = SVC()
svm.fit(x_train.values, y_train.values)
svm_predicts = svm.predict(x_test.values)

CM(svm_predicts, y_test)

array([[ 8,  5],
       [18, 24]])

## Decision Tree

In [32]:
dt = DTC()
dt.fit(x_train.values, y_train.values)
dt_predicts = dt.predict(x_test.values)

CM(dt_predicts, y_test)

#plot_tree(dt)

array([[24,  5],
       [ 2, 24]])

## Random Forest

In [33]:
rf = RFC()
rf.fit(x_train.values, y_train.values)
rf_predicts = rf.predict(x_test.values)

CM(rf_predicts, y_test)

array([[22,  2],
       [ 4, 27]])

Vizualizacão